
# nb 05: implement / use (skill-) action-services

**This is the continuation of the tutorial 01-04 notebooks. Please understand how to read data and send simple commands from the cobot before continuing here
You can also find additional information about underlying docus and startup procedures**

**CHANGELOG**

v0.3

- imports changed


## Imports / Packages needed

In [ ]:
# packages needed for this tutorial
# math and robotics
import numpy as np
import quaternion
import spatialmath as sm
from tqdm.notebook import tqdm, trange

# plotting 
import seaborn as sns
import matplotlib.pylab as plt

# ROS
import rospy

# hrr-cobot packages
import hrr_cobot_robot

# set printing and plotting options    
np.set_printoptions(precision=5, suppress=True)
sns.set_theme('notebook')
%matplotlib inline

###### connect to ROS 

optionally set the ```ROS_IP``` and ```ROS_MASTER_URI``` directly form this notebook, as
the setup above spawns the ros-setup from the real-time client, i.e. ```hrrN3511rt2004.lsr.ei.tum.de```

In [ ]:
def get_hostname_and_IP():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_DGRAM) as s:
        s.connect(("8.8.8.8", 80))
        return socket.gethostname(), s.getsockname()[0]

_host, _ip = get_hostname_and_IP()
%env ROS_MASTER_URI=http://hrrcobotLinux54.lsr.ei.tum.de:11311
# %env ROS_MASTER_URI=http://localhost.lsr.ei.tum.de:11311
print(f"\t\033[1m\033[4m!!!please check that the IPs below is identical!!!\033[0m"+
      f"\n\thost {_host} has IP {_ip}, which should be identical to",end="\nvs:\t")
%env ROS_IP=$_ip

In [ ]:
rospy.init_node("hrr_cobot_tutorial05")

In [ ]:
hrr_cobot_robot.utils.load_default_parameters("/hrr_cobot")
cobot = hrr_cobot_robot.HrrCobotControl.from_ros(prefix="/hrr_cobot")

In [ ]:
cobot.reset()
cobot.run_calibration_routine(file_name='/tmp/current_tool.npy', plot=True)

In [ ]:
cobot.B_err_F

In [ ]:
print(cobot)

## HRR-cobot Observer Demo

The observer handle is just a useful handle to record data online without the need of additional ros-bag parsing

In [ ]:
observer = hrr_cobot_robot.HrrCobotObserver(1000, "q", "B_F_msr", "sns_pos", "B_err_F", "B_F_ext")
observer.drop(cobot);

In [ ]:
observer.reset()
# cobot.reset()
# cobot.move_to_pose(sm.SE3(0.1, 0.0, -0.2) @ cobot.FK(cobot.q_calib))
T = 200
for t in trange(T):
    cobot.update()
    observer.update(cobot)
cobot.emergency_stop()
print("\n".join([f"Stored {k}\t∈ {v.shape}" for k,v in observer.drop(cobot).items()]))

In [ ]:
data = observer.drop(cobot)

In [ ]:
x = np.linspace(0, T * cobot.dt, T) * 1e3
plt.plot(x, data["q"] - data["q"][0, :]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'$\Delta{\bf q}$ [rad]');
plt.legend([f"$\Delta q_{i}$" for i in range(6)], ncol=3);

In [ ]:
plt.plot(x, data["sns_pos"] - data["sns_pos"][0,:]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'$\Delta{\bf p}_{\mathrm{sns}}$ [m]');
plt.legend(["$\Delta x$", "$\Delta y$", "$\Delta z$"]);

In [ ]:
plt.plot(x, data["B_F_msr"][:, :3]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'${}^{B}{\bf F}_{\mathrm{msr}}$ [N]');
plt.legend([f'$F_{{{x}}}$' for x in ["x", "y", "z"]]);

### Plot Compliant Behavior

Refer to Tutorial 4 for setup / configuration

* zero-force as desired value (compliance)
* activate axes as needed (here compliance along xyz)


In [ ]:
observer.reset()
cobot.init_sns_vel()
cobot.reset()
cobot.set_py_hybrid_force_vel_command(B_F_des=np.r_[np.zeros(2), 0.0, np.zeros(3)], 
                                      K_f=1e-3, K_t=2e-3,
                                      wrench_dir=np.r_[1.0, 1.0, 1.0, 0.0, 0.0, 0.0],
                                      scale_pos=0.0, scale_rot=0.0, 
                                      vel_dir=np.zeros(6))

In [ ]:
T = 10000
u_cmd = np.zeros((T, 6))
for t in trange(T):
    cobot.update_tf()
    cobot._hybrid_force_vel_command()
    observer.update(cobot)
    u_cmd[t, 0:3], u_cmd[t, 3:6] = cobot._F_ctrl.u_F(cobot.B_err_F, cobot.R_B_C)
cobot.emergency_stop()
print("\n".join([f"Stored {k}\t∈\t{v.shape}" for k,v in observer.drop(cobot).items()]))

In [ ]:
data = observer.drop(cobot)

In [ ]:
T = data["B_F_msr"].shape[0]
x = np.linspace(0, T * cobot.dt, T) * 1e3
# F-msr
plt.figure("F_msr")
plt.plot(x, data["B_F_msr"]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'${}^{B}{\bf F}_{\mathrm{msr}}$ [N, Nm]');
# F-ext
plt.figure("B_F_ext")
plt.plot(x, data["B_F_ext"]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'${}^{B}{\bf F}_{\mathrm{ext}}$ [N, Nm]');
# F-err
plt.figure("B_err_F")
plt.plot(x, data["B_err_F"]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'${}^{B}{\bf \varepsilon}_{\mathrm{F}}$ [N, Nm]');
# u-cmd
plt.figure("u")
plt.plot(x, u_cmd[:T, 0:3]);
plt.xlabel(r'$t$ [ms]')
plt.ylabel(r'${}^{B}{\bf u}_{\mathrm{pos}}$ [m/s]');

In [ ]:
cobot._F_ctrl.S_diag

## Action Service Example

> **NOTE: this requires hrr_cobo_robot to be of version v0.1.4+**

useful ROS-tutorials: 

- [Writing a Simple Action Server using the Execute Callback (Python)](http://wiki.ros.org/actionlib_tutorials/Tutorials/Writing%20a%20Simple%20Action%20Server%20using%20the%20Execute%20Callback%20%28Python%29)
- [Writing a Simple Action Client (Python)](http://wiki.ros.org/actionlib_tutorials/Tutorials/Writing%20a%20Simple%20Action%20Client%20%28Python%29)

Below we communicate with an example action-service that uses the existing API,
namely the ```CalibrationServer```,
which can be started as a standalone ROS-node via 

```
rosrun hrr_cobot_robot ft_calibrator
```

which will run the ``ft_sensor_calibrator`` from the ``ros_nodes_click_commands.py`` file.

It is best to start this function from a programming IDE (pycharm / vscode / spider) and run debug-breakpoints.

This notebook can then be used to initiate / trigger the action-service as outlined below

### Import required ROS-(action)-messages

In [ ]:
from hrr_msgs.msg import CalibrateCobotAction, CalibrateCobotGoal
import actionlib

### search for dedicated action message
simple execute the code below.

By the time of writing, this was given as 

```bash
rostopic list | grep "calibrate/goal"
/ft_sensor_calibrated/calibrate/goal
```

In [ ]:
!rostopic list | grep "calibrate/goal"

### set up action-client

given the message above, the topic of interest woudld be "/ft_sensor_calibrated/calibrate/"

In [ ]:
calibrate_cobot = actionlib.SimpleActionClient("/ft_sensor_calibrated/calibrate", CalibrateCobotAction)
calibrate_cobot.wait_for_server()

### send goal to cobot

It is recommended to first inspect the goal and then send it

In [ ]:
goal = CalibrateCobotGoal()

In [ ]:
calibrate_cobot.send_goal(goal)